## Imports/Aux

In [ ]:
# colab only

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# colab only

%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project
! pip3 install -r requirements.txt

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.6/269.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 54.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 14.3 MB/s eta 0:00:00
  Using cached numpy-1.24.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.0/769.0 kB 47.5 MB/s et

In [8]:
%cd /content/gdrive/MyDrive/nlp-final/
%ls
%pwd
#! git clone https://github.com/moses-smt/mosesdecoder.git
%env MOSES=/content/gdrive/MyDrive/nlp-final/mosesdecoder

%env FASTBPE=/content/gdrive/MyDrive/nlp-final/fastBPE
%cd fastBPE
! g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

/content/gdrive/MyDrive/nlp-final
fairseq/  fastBPE/  mosesdecoder/  NLP-Final-Project/
env: MOSES=/content/gdrive/MyDrive/nlp-final/mosesdecoder
env: FASTBPE=/content/gdrive/MyDrive/nlp-final/fastBPE
/content/gdrive/MyDrive/nlp-final/fastBPE


In [ ]:
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/utils

! bash preprocess.sh

In [4]:
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/utils
! bash train.sh

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/utils
2023-04-20 21:37:50.800205: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 21:37:52.978465: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-20 21:37:56 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-04-20 21:37:57 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-04-20 21:37:59 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1

In [4]:
# colab only

import sys
sys.path.append('/content/drive/MyDrive/NLP_Final_Project/src')

In [10]:
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src

data_dir = '/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/relis-bin'
model_dir = '/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/checkpoints/RE-WhiteText-BioGPT'
model_file = 'checkpoint_avg.pt'
src_file = '/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/relis_test.tok.bpe.x'
output_file = 'generate_' + model_file
beam = 1
decoding_length = 1024

! python utils/average_checkpoints.py --inputs='/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/checkpoints/RE-WhiteText-BioGPT' --output='/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/checkpoints/RE-WhiteText-BioGPT/checkpoint_avg.pt' --num-epoch-checkpoints=2

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src
2023-04-20 22:08:37.255050: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 22:08:38.334852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-20 22:08:40 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Namespace(inputs=['/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/checkpoints/RE-WhiteText-BioGPT'], output='/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/checkpoints/RE-WhiteText-BioGPT/checkpoint_avg.pt', num_epoch_checkpoints=2, num_update_checkpoints=None, checkpoint_upper_bound=None)
averaging checkpoints:  ['/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/che

## Model Setup

In [16]:
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src

from src.transformer_lm_prompt import TransformerLanguageModelPrompt

src_inputs = []
with open(src_file) as reader:
    for line in reader:
        src_inputs.append(line.strip())
    
try:
  m = TransformerLanguageModelPrompt.from_pretrained(
      model_dir, 
      model_file, 
      data_dir,
      max_len_b=decoding_length,
      max_tokens=12000,)
except AssertionError as e:
  print(e)

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src
